# **Heart Disease Diagnostic Tool**

## **Introduction**

Cardiovascular diseases are the leading cause of death globally, estimated to take around 17.9 million lives each year. It is a group of disorders of the heart and blood vessels and include coronary heart disease, cerebrovascular disease, rheumatic heart disease and other conditions (WHO, n.d.). For our project, the question we are trying to answer is:

“Can we use patient attributes from a clinic's database to predict whether a patient has heart disease?”

The data set used to answer this questions was built using 303 patient information from Cleveland and was processed to include 14 out of 76 attributes most commonly used by published experiments. It was obtained from https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data (UCI Machine Learning Repository, n.d.).


## **Preliminary exploratory data analysis**

In [ ]:
library(rvest)
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

#### **Reading and Wrangling the Data**

We set the seed for reproducibility and read the data from the web into an R dataframe.   

The data has no column names, but using the information from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Heart+Disease) we give the dataframe its appropriate column names. We also convert the variables into numeric and categorical based on the above documentation.

In [ ]:
set.seed(293)

dataset_test <- download.file(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", 
    "heart_disease.csv")

heart_data <- read_csv("heart_disease.csv", col_names = FALSE)


heart_data <- rename(heart_data, 
                     age =X1, #represented in years - numerical
                     sex =X2, #1 representing male; 0 representing female - categorical
                     cp =X3, #Chest pain type - categorical 
                     trestbps =X4, #resting blood pressure (mmHg) - numerical
                     chol = X5, #serum cholesterol (mg/dl) - numerical
                     fbs =X6, #fasting blood sugar > 120 mg/dl (1 = true; 0 = false) - categorical
                     restecg=X7, #resulting electrocardiographic results - categorical
                     thalach=X8, #maximum heart rate achieved - numerical
                     exang=X9, #exercise induced angina - categorical
                     oldpeak=X10, #slope of peak exercise ST segment - categorical
                     slope=X11, #slope of peak exercise ST segment - categorical
                     ca=X12, #number of major vessels - categorical
                     thal=X13, #thalassemia Severity - categorical
                     diagnosis=X14) #diagnosis of heart disease - categorical

idx <- heart_data == "?"
is.na(heart_data) <- idx                                 #eliminates ? and replaces with NA

heart_data[] <- sapply(heart_data, as.numeric) #converts all the data to numeric

names = c('sex', 'cp', 'exang', 'slope', 'thal', 'diagnosis', "restecg", "ca", "fbs")
heart_data[,names] <- lapply(heart_data[,names] , factor)

heart_data <- mutate(heart_data, diagnosis = ifelse(diagnosis == "1"|diagnosis == "2"|diagnosis == "3"|diagnosis == "4", "Positive", "Negative"))
heart_data <- mutate(heart_data, diagnosis = as.factor(diagnosis))          #changes the diagnosis values to strings

heart_data


**Table 1:** Data from the Cleveland clinic, showing patient attributes and heart disease diagnoses. 

## **Methods & Results**

#### **Splitting data into training and testing sets**

We will be using K-nearest neighbours to solve our classification question, so we must split our data into a training set and a testing set. Training set will be used for building the classifier while the testing set will be used for evaluating accuracy of the classifier. 

In [ ]:
heart_split <- initial_split(heart_data, prop = 0.75, strata = diagnosis)          #splits data
heart_train <- training(heart_split)
heart_test <- testing(heart_split) 

Later on we will perform a 5-fold cross-validation to select the number of neighbours that gives the highest accuracy for our classifier.

#### **Number of Positive and Negative diagnoses**

Next, we will create useful summarizations of our data to select suitable predictor variables.  

One useful summary of the data is the number of positive and negative diagnosis in our training data. We will calculate the total number of observations in our training data to calculate the proportion of positive and negative diagnosis represented as a percentage.

In [ ]:
total_count <- nrow(heart_train)
count_train_summary <- group_by(heart_train, diagnosis) |>    #counts number of positive/negative diagnosis
    summarize(count=n()) |>
    mutate(percentage = count/total_count)

count_train_summary

**Table 2:** Counts and percentages of negative and positive heart disease cases in the training set.

From the training dataset we can see there's a good representation of patients with (45.8%) and without (54.2%) heart disease which is good, as later on we will know if our diagnosis tool is actually making accurate predictions instead of always predicting postive/negative. Since negative diagnosis represents the majority of the training data, the majority classifier would always predict that a new observation is negative so the estimated accuracy of the majority classifier would be 54.2%. The accuracy of the majority classifier can be later used to evaluate the accuracy of our classifier.

#### **Number of missing values in each column**

We decided to avoid using ***categorical*** variables as our predictors because our goal is to perform classification using K-nearest neighbors, which would work better with numerical predictors. 

So, we created a table that summarizes the number of missing values for all ***numerical*** variables.

In [ ]:
heart_data |>
    select(age, trestbps, chol, thalach, oldpeak, diagnosis) |>
    summarise_all(funs(sum(is.na(.))))

**Table 3:** Summary of missing values of numerical variables.

All variables of interest have no missing values, so no more data processing needs to be done.

#### **Average of numerical columns**

Another useful summarization of the data is the average value of each variable, across both positive and negative diagnoses.

In [ ]:
stats_avg <- heart_train |>
    select(age, trestbps, chol, thalach, oldpeak, diagnosis) |>
    group_by(diagnosis) |>
    summarize(across(age:oldpeak, mean, na.rm = TRUE))

stats_avg

**Table 4:** Summary of average variables for positive and negative diagnoses.

We see that age, chol, thalach, and oldpeak have the most significant difference between positive and negative diagnoses so we will create visualizations to choose two variables out of them to use in our model.

#### **Visualizations of Predictor Variables**

From the above summarizations, we were able to narrow down our list of candidate predictors. Let's go even further.

We will make histogram plots of age, chol, thalach, and oldpeak to see the distribution of each of the potential predictor variables.

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 20)
library(gridExtra)

age_plot <- heart_train |>
        ggplot(aes(x = age, fill = diagnosis)) +
        geom_histogram() +
        labs(x = "Age (years)", y = "number of patients", fill = "Heart Disease Diagnosis") + 
        ggtitle("(Fig. 1a) Histogram of age based on Heart Disease status") +
        theme(text = element_text(size = 15)) +
        facet_grid(rows = vars(diagnosis))

chol_plot <- heart_train |>
        ggplot(aes(x = chol, fill = diagnosis)) +
        geom_histogram() +
        labs(x = "Cholesterol in serum (mg/dl)", y = "number of patients", fill = "Heart Disease Diagnosis") + 
        ggtitle("(Fig. 1b) Histogram of cholesterol level based on Heart Disease status") +
        theme(text = element_text(size = 15)) +
        facet_grid(rows = vars(diagnosis))


thalach_plot <- heart_train |>
        ggplot(aes(x = thalach, fill = diagnosis)) +
        geom_histogram() +
        labs(x = "Maximum Heart Rate (bpm)", y = "number of patients", fill = "Heart Disease Diagnosis") + 
        ggtitle("(Fig. 1c) Histogram of max heart rate based on Heart Disease status") +
        theme(text = element_text(size = 15)) +
        facet_grid(rows = vars(diagnosis))


oldpeak_plot <- heart_train |>
        ggplot(aes(x = oldpeak, fill = diagnosis)) +
        geom_histogram() +
        labs(x = "ST depression induced by exercise relative to rest", y = "number of patients", fill = "Heart Disease Diagnosis") + 
        ggtitle("(Fig. 1d) Histogram of oldpeak based on Heart Disease status") +
        theme(text = element_text(size = 15)) +
        facet_grid(rows = vars(diagnosis))


grid.arrange(age_plot, chol_plot, thalach_plot, oldpeak_plot, ncol = 2)

**Figure 1(a-d):** Histograms of further narrowed down numerical predictor variables, for positive and negative heart disease diagnoses (1a is age, 1b is cholesterol, 1c is max heart rate, 1d is oldpeak).

These histograms show the distribution of patients that have negative and positive diagnosis for the four variables we are exploring.

The oldpeak distribution stands out the most because of the large number of zero values. All of this overlap in values gives us an inclination that oldpeak might not be suitable to use as a predictor.    
  


Next, we will plot each variable against every other variable in scatterplots to see if we can find any correlations in a 2D visualization.

In [ ]:
options(repr.plot.height = 5, repr.plot.width = 7)

age_thalach_plot <- heart_train |>
    ggplot(aes(x=age, y=thalach, color= diagnosis))+
    geom_point()+
    labs(x="Age", y="Maximum Heart Rate (bpm)", color="Heart Disease Diagnosis") +
    theme(text = element_text(size = 15)) +
    ggtitle("(Fig. 2a) Maximum Heart Rate (Thalach) vs Age")
age_thalach_plot

age_chol_plot <- heart_train |>
    ggplot(aes(x=age, y=chol, color= diagnosis))+
    geom_point()+
    labs(x="Age", y="Cholesterol in serum (mg/dl)", color="Heart Disease Diagnosis") +
    theme(text = element_text(size = 15)) +
    ggtitle("(Fig. 2b) Cholesterol Levels vs Age")
age_chol_plot

age_oldpeak_plot <- heart_train |>
    ggplot(aes(x=age, y=oldpeak, color= diagnosis))+
    geom_point()+
    labs(x="Age", y="Oldpeak", color="Heart Disease Diagnosis") +
    theme(text = element_text(size = 15)) +
    ggtitle("(Fig. 2c) Oldpeak vs Age")
age_oldpeak_plot

chol_thalach_plot <- heart_train |>
    ggplot(aes(x=chol, y=thalach, color= diagnosis))+
    geom_point()+
    labs(x="Cholesterol in serum (mg/dl)", y="Maximum Heart Rate (bpm)", color="Heart Disease Diagnosis") +
    theme(text = element_text(size = 15)) +
    ggtitle("(Fig. 2d) Maximum Heart Rate (Thalach) vs Cholesterol Levels")
chol_thalach_plot

chol_oldpeak_plot <- heart_train |>
    ggplot(aes(x=chol, y=oldpeak, color= diagnosis))+
    geom_point()+
    labs(x="Cholesterol in serum (mg/dl)", y="Oldpeak", color="Heart Disease Diagnosis") +
    theme(text = element_text(size = 15)) +
    ggtitle("(Fig. 2e) Oldpeak vs Cholesterol Levels")
chol_oldpeak_plot

oldpeak_thalach_plot <- heart_train |>
    ggplot(aes(x=thalach, y=oldpeak, color= diagnosis))+
    geom_point()+
    labs(x="Maximum Heart Rate (bpm)", y="Oldpeak", color="Heart Disease Diagnosis") +
    theme(text = element_text(size = 15)) +
    ggtitle("(Fig. 2f) Oldpeak vs Maximum Heart Rate (Thalach)")
oldpeak_thalach_plot


**Figure 2(a-f):** Scatterplots of maximum heart rate vs age (2a), cholesterol vs age (2b), oldpeak vs age (2c),  max heart rate vs cholesterol (2d), oldpeak vs cholesterol (2e), and oldpeak vs max heart rate (2f).

From graphs (2c), (2e), and (2f), we see that many observations had an oldpeak of zero, causing lots of overlap. It's pretty clear that the k-nn algorithm would not be happy with oldpeak as a predictor. This confirms our suspicion that we shouldn't be using the `oldpeak` variable.

Looking at the remaining three graphs (2a), (2b), and (2d) plotting age, serum cholesterol and thalach against each other, we decided that `age` and `thalach` are the best variables to use in our model because the graph for thalach versus age (2a) shows more distinct regions of positive and negative heart disease diagnoses which is useful for K-nearest neighbour classification. For the other two graphs (2b) and (2d), the pattern for positive and negative heart diagnoses isn't as clear and data points for positive and negative diagnoses are more interspersed within one another. 


### **Data Analysis**

Now that we have our two predictors, `age` and `thalach`, we're one step closer to predicting the classification of a new heart disease patient. 

To accomplish our goal, we will first perform a 5-fold cross-validation using the `vfold_cv` function set to `v = 5`. We can create a K-nearest neighbor model specification, setting neighbors to tune, followed by a workflow analysis computing accuracies for k values from 1 to 25. 

In [ ]:
set.seed(293)

heart_recipe <-recipe(diagnosis ~ age + thalach, data = heart_train) |>   #creates a recipe containing oldpeak+thalach
    step_scale(all_predictors()) |>                                       #as our predictors, as well as scaling them.                         
    step_center(all_predictors())

## Cross Validation 
# using 10 folds
heart_vfold <- vfold_cv(heart_train, v = 5, strata = diagnosis)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

k_vals <- tibble(neighbors = seq(from = 1, to = 25, by = 1))

knn_results <- workflow() |>
  add_recipe(heart_recipe) |>
  add_model(knn_spec) |>
    tune_grid(resamples = heart_vfold, grid = k_vals) |>
    collect_metrics()

knn_results

**Table 5:** K-nearest neighbours unfiltered accuracy results for different K neighbours using 5-fold cross validation.

We can then filter these results for the accuracy, graph a line plot with neighbours on the x-axis and the mean on the y-axis, and extract the k value that corresponds to the highest accuracy.

In [ ]:
#compute accuracy for every fold
accuracies <- knn_results |>
  filter(.metric == "accuracy")
accuracies

**Table 6:** Filtered accuracy for K-nearest neighbours using 5-fold cross validation.

In [ ]:
#get row (and K value) that corresponds to the highest accuracy
best_k <- accuracies |>
    filter(mean == max(mean)) |>                
    select(neighbors) |>
    pull()
best_k


#plot of Accuracy vs K
accuracy_plot <- accuracies |>
    ggplot(aes(x = neighbors, y = mean))+
    geom_point()+
    geom_line() +
    labs(x = "Number of Neighbours (K)", y = "Accuracy Estimate (mean)") + 
    ggtitle("(Fig. 3) Number of Neighbours and Accuracy Estimate") +
    theme(text = element_text(size = 15))
accuracy_plot

print(filter(accuracies, neighbors == best_k))               #prints the accuracy of a model with best k

**Figure 3:** Plot of accuracy estimate versus many number of neighbours values.

We can see from the graph that the highest accuracy from our cross-validation tests corresponds to a k value of 19 neighbours, with an accuracy of ~71.6%. Therefore we will now create a K-nearest neighbour model using the number of neighbours set to 19.

In [ ]:
## KNN model creation using our optimal K value calculated from above

knn_spec2 <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k) |>   
    set_engine("kknn") |>
    set_mode("classification")

heart_fit <- workflow() |>
    add_recipe(heart_recipe) |>
    add_model(knn_spec2) |>
    fit(data = heart_train)

Now, let's test this model against our test data to evaluate the performance of our model. We can also make a heat map that visualizes our k-nn algorithm.

In [ ]:
## test our model with the test data
heart_test_predictions <- predict(heart_fit, heart_test) |>
    bind_cols(heart_test)

# heart_test_predictions

## evaluate accuracy of our model using test data
heart_prediction_accuracy <- heart_test_predictions |>
    metrics(truth = diagnosis, estimate = .pred_class) |>
    filter(.metric == "accuracy")

heart_prediction_accuracy

**Table 7:** Accuracy of the KNN classifcation model on the testing set

In [ ]:
heart_mat <- heart_test_predictions |>
    conf_mat(truth = diagnosis, estimate = .pred_class)
heart_mat

**Table 8**: Confusion matrix comparing true heart disease diagnoses to predicted diagnoses.

In [ ]:
are_grid <- seq(min(heart_data$age), 
                max(heart_data$age), 
                length.out = 100)
smo_grid <- seq(min(heart_data$thalach), 
                max(heart_data$thalach), 
                length.out = 100)
asgrid <- as_tibble(expand.grid(age = are_grid, 
                                thalach = smo_grid))

# use the fit workflow to make predictions at the grid points
knnPredGrid <- predict(heart_fit, asgrid)

# bind the predictions as a new column with the grid points
prediction_table <- bind_cols(knnPredGrid, asgrid) |> 
  rename(diagnosis = .pred_class)


## Creates a heat map of regions depicting a visual representation of how our classifier algorothim picks diagnosis.
wkflw_plot <- heart_train |>
  ggplot() +
  geom_point(mapping = aes(x = age, 
                           y = thalach, 
                           color = diagnosis), 
             alpha = 0.75) +
  geom_point(data = prediction_table, 
             mapping = aes(x = age, 
                           y = thalach, 
                           color = diagnosis), 
             alpha = 0.02, 
             size = 5) +
  labs(color = "Diagnosis", 
       x = "Age", 
       y = "Thalach") +
  scale_color_manual(labels = c("Positive", "Negative"), 
                     values = c("orange2", "steelblue2")) +
  theme(text = element_text(size = 15)) +
    ggtitle("(Fig. 4) Training Data with Background Colour\nDepicting Classifier Decision")

wkflw_plot

**Figure 4:** Heat map visualization of knn classification using age and thalach (maximum heart rate) as predictors.  Yellow is where positive diagnoses are predicted and blue is negative diagnoses are predicted.

## **Discussion**

We found that our classifier has a maximum of 71.6% accuracy after performing a 10-fold cross-validation and choosing the number of neighbours to be 19. When testing our classifier against our testing dataset, we got a 67.1% accuracy. Looking at the confusion matrix, we have 25 diagnoses that were negative which we predicted accurately and 26 diagnoses that were positive which we predicted accurately. However, 9 positive diagnoses were predicted to be negative and 16 negative diagnoses were predicted to be positive. Since predicting a negative diagnosis to be positive is less severe than predicting a positive diagnosis to be negative, we have 9 out of 76 cases (11.8%) that would result in severe consequences. We also created a scatter plot of thalach versus age where the background colour indicates the decision of the classifier.

We intially set out to determine if we could create a classificiation algorithim based on K-nearest neighbours using attributes from the cleveland heart disease data set. Our expectation was to create a classifier that had at least 90% accuracy so that we can potentially use it for predicting heart disease. However, after visualizing the data, we determined that the best attributes to use as predictors were `age` and `thalach`. The K-nearest neighbour classification model we made resulted in a 67.1% accuracy against new data, which is significantly lower than what we had hoped for. With that being said, our classifier is still slightly better than making random diagnosis aligning with our expectations based off the histogram and scatter plot analysis. Also, the accuracy of our classifier is around 13% higher than the majority classifier calculated in the "Methods and Results" section which has an accuracy of 54.2% so our method is extracting some useful information from our predictor variables. However, due to the importance of making an accurate diagnosis this low accuracy level is not suitable for our purposes in predicting heart disease. The low accuracy of our classifier might be improved by using a larger dataset so that we have more training data to train our classifier resulting in improved accuracy.

Our findings show that using the two attributes, `thalach` (max heart rate) and `age`, alone is unlikely sufficient enough to produce an accurate K-nearest neighbour model. However, since our results show a strong correlation between maximum heart rate and age with increased risk of heart disease. Perhaps under a different prediction model and/or with more prediction factors, maximum heart rate and age can still be used as important attributes to look out for when assessing heart disease risk. Or doctors could look at maximum heart rate and age as stronger indicators/risk factors when assessing heart disease risk in patients. 

The mortality rate of heart diseases can be reduced if the diseases can be accurately predicted at early stages and preventative measures are implemented right away (Devi et al., 2021). A future question our project leads to is if a different selection of attributes or a different prediction model be used to predict heart disease in patients. Since our project was based soley off the Cleveland dataset, future projects from datasets like the ones from Hungary and Switzerland can be used to test if other attributes are better indicators of heart disease or if the attributes we used (`thalach` and `age`) are less accurate in the different datasets.

## **References**

Devi, A. G., Borra, S. P. R., & Sagar, K. V. (2021). A Method of Cardiovascular Disease Prediction using Machine Learning. IJERT. Retrieved Arpil 9, 2023, from https://doi.org/10.17577/IJERTCONV9IS05050

Heart Disease Data Set. (n.d.). UCI Machine Learning Repository. Retrieved March 11, 2023 from https://archive.ics.uci.edu/ml/datasets/Heart+Disease

World Health Organization. (n.d.). Cardiovascular diseases. World Health Organization. Retrieved March 11, 2023, from https://www.who.int/health-topics/cardiovascular-diseases#tab=tab_1 